# M-HITE

In [1]:
# Import libraries
import pandas as pd
import math

In [2]:
# Define constants & variables
## Common physical constants, mainly used to calculate fMax (the upper bound of energy flux received by a planet ...
## ... that allows it to be habitable)
BIGG       = 6.67428e-11       # gravitational constant
PI         = 3.1415926535
A          = 0.7344            # Pierrehumbert's Constant
SB         = 5.670373e-8       # Stefan-Boltzmann Constant
LH2O       = 2.425e6           # Latent Heat Capacity of Water
RGAS       = 461.5             # Universal Gas Constant
PLINE      = 1e4               # 
PREF       = 610.616           # Reference Pressure
TREF       = 273.13            # Reference Temperature
K0         = 0.055             # A constant in Runaway Greenhouse calculation

## 
ALBMINELSE = 0.05
ALBMAXELSE = 0.8
ALBMING    = 0.25
ALBMAXM    = 0.35
MINFLUX    = 67

## Defintion of units of measurement, mainly used to convert exoplanet.org data into SI units
MEARTH     = 5.972186e24       # Earth mass in kilograms
REARTH     = 6378100           # Earth's radius in meters
S0         = 1362              # Solar constant in watts per square meter
MSUN       = 1.988416e30       # Solar mass in kilograms
RSUN       = 6.957e8           # Solar radius in meters
LSUN       = 3.828e26          # Solar luminosity in watts
RJUP       = 7.1492e7          # Jovian radius in meters
MJUP       = 1.8982e27         # Jovian mass in kilograms
AU         = 1.496e11          # The astronomical unit in meters

In [3]:
#### Define functions

### Probability distribution of eccentricity (from the original code, not used in M-HITE)
### pofe = probability of e
### def pofe(ecc):
    #### return 0.1619 - 0.5352*ecc + 0.6358*ecc*ecc - 0.2557*ecc**3

### Probability distribution of eccentricity (uses data from the exoplanet data file)
def mpofe(ecc,mu,sigma):
    return ((sigma*math.sqrt(2*math.pi))**(-1))*math.exp(-(((ecc-mu)**2)/(2*sigma**2)))/1000

### Zeng-Sasselov boundaries (digunakan oleh fpRocky)
def mzsrank(mPlanet,mZSi,mZSimin1):
    w = (mZSi - mZSimin1)/10
    i = 1
    while i < 10:
        mTest = mZSimin1 + i*w
        if mPlanet < mTest:
            rank = i
            # print("rank: ",rank,"; mPlanet: ",mPlanet,"; mTest: ",mTest, "; i:", i)
            break
        else:
            rank = i
            i = i + 1
    return rank

### Calculate probability of planet's rocky-ness/terrestriality
def fpRocky (mPlanet,rPlanet,exoName):
    mPlanet = mPlanet/MEARTH # Convert the unit to Earth's masses
    rPlanet = rPlanet/REARTH # Convert the unit to Earth's radii
    
    # Calculate mu1
    mu1 = 0.0
    mZSimin1 = 0
    rZSimin1 = 0
    for i in zsList:
        mZSi = zs.loc[i, "M-PureMgSiO3"]
        rZSi = zs.loc[i, "R-PureMgSiO3"]
        if mPlanet == mZSi:
            mu1 = rZSi
            break
        elif mZSi > mPlanet:
            mu1 = rZSimin1 + mzsrank(mPlanet,mZSi,mZSimin1)*(rZSi-rZSimin1)/10
            break
        else:
            mZSimin1 = mZSi
            rZSimin1 = rZSi # the loop won't stop until either of the two conditions above are satisfied 
    print("mu1: ", mu1)

    ### Calculate mu2
    mu2 = 0.0
    mZSimin1 = 0
    rZSimin1 = 0
    for i in zsList:
        mZSi = zs.loc[i, "M-MgSiO3-H2O-5050"]
        rZSi = zs.loc[i, "R-MgSiO3-H2O-5050"]
        if mPlanet == mZSi:
            mu2 = rZSi
            break
        elif mPlanet < mZSi:
            mu2 = rZSimin1 + mzsrank(mPlanet,mZSi,mZSimin1)*(rZSi-rZSimin1)/10
            break
        else:
            mZSimin1 = mZSi
            rZSimin1 = rZSi
    print("mu2: ", mu2)

    ### Calculate sigma1
    sigma1 = (mu2-mu1)/3
    print("sigma: ",sigma1)
    
    pRocky = 0
    if rPlanet <= mu1:
        pRocky = 1
    elif rPlanet >= mu2:
        pRocky = 0
    else: # use the T_M_p function from SEPHI
        pRocky = math.exp(-(0.5)*((rPlanet-mu1)/sigma1)**2)
        
    return pRocky


In [4]:
### Import exoplanet data from a CSV into a pandas dataframe
exo = pd.read_csv (r'exoplanetsInUse_noKOI1.csv', low_memory=False)

### Set the column with the header NAME to be used as an index to identify row 
exo = exo.set_index("NAME", drop = False)

### Extract names of planets as a list (to be used as a calling list)
exoList = pd.DataFrame(exo, columns=['NAME'])
exoList = exoList['NAME'].values.tolist()

In [5]:
### Import CSV of Zeng & Sasselov boundaries
zs = pd.read_csv (r'zeng-sasselov_boundaries.csv')

### Set index using the SORT column
zs = zs.set_index("SORT", drop = False)

### Extract the column "SORT" as a list (to be used as a calling list)
zsList = pd.DataFrame(zs, columns=['SORT'])
zsList = zsList['SORT'].values.tolist()

In [6]:
## Subroutine to determine Habitability Index value

habIndexList = []

for exoName in exoList:
    #Extract data of individual planets
    
    #### HOST STAR PROPERTIES
    ### Stellar radius (in solar radii)
    rStar = exo.loc[exoName, "RSTAR"]
    # Convert to SI
    rStar = rStar*RSUN
    ### Stellar temperature (in Kelvin)
    teffStar = exo.loc[exoName, "TEFF"]
    ### Stellar luminosity
    luminosity = 4*math.pi*rStar*rStar*SB*teffStar**4
    
    ###### PLANET PROPERTIES   
    ### Planetary radius (in Jovian radii)
    rPlanet = exo.loc[exoName, "R"]
    # If Rp is not available, calculate it from  transit depth
    if math.isnan(rPlanet) == 1:
        depth = exo.loc[exoName, "DEPTH"]
        rPlanet = math.sqrt(depth)*rStar
    # Convert to SI
    rPlanet = rPlanet*RJUP
    ### Planetary mass (in Jovian masses)
    mPlanet = exo.loc[exoName, "MASS"] 
    # If Mp is not available, calculate it from a common scaling law, [...]
    # using Rp
    if math.isnan(mPlanet) == 1:
        if rPlanet/REARTH <= 1:
            mPlanet = ((rPlanet/REARTH)**3.268)*MEARTH
        elif rPlanet/REARTH > 1:
            mPlanet = ((rPlanet/REARTH)**3.65)*MEARTH
    # Convert to SI
    mPlanet = mPlanet*MJUP
    ### Surface planet gravity (in SI)
    surfGrav = BIGG*mPlanet/(rPlanet**2)    
    
    
    
    ###### Orbital properties
    ### Orbital eccentricity
    ecc = exo.loc[exoName, "ECC"]
    ### Measurement uncertainty of orbital eccentricity    
    # Upper bound (relative from E)
    eccUpRel = exo.loc[exoName, "ECCUPPER"]
    # If measurement uncertainty is not available, assign it as 0.01
    if math.isnan(eccUpRel) == 1:
        eccUpRel = 0.01
    # Upper bound (relative from E)
    eccUpper = ecc + eccUpRel
    # Lower bound (relative from E)
    eccLowRel = exo.loc[exoName, "ECCLOWER"]
    # If measurement uncertainty is not available, assign it as 0.01
    if math.isnan(eccLowRel) == 1:
        eccLowRel = 0.01
    # Lower bound (absolute)
    eccLower = ecc - eccLowRel
    ### Orbital semi-major axis (in AU)
    semiAxis = exo.loc[exoName, "A"]      
    # Convert to SI
    semiAxis = semiAxis*AU
    
    
    ###### Calculate the upper and lower bounds of F_OLR [...]
    ###### that would allow for surface liquid water to exist
    ### lupa apa
    pStar = PREF*math.exp(LH2O/(RGAS*TREF))
    ### Maximum F_OLR
    fMax = A*SB*(LH2O/(RGAS*math.log(pStar*math.sqrt(K0/(2*PLINE*surfGrav)))))**4
    
    ### Maximum F_OLR (Fmin) is the constant MINFLUX

    
    ###### Probability of rocky-ness (new)
    pRocky = fpRocky(mPlanet,rPlanet,exoName)
        
    
    ###### Albedo (new)
    ### Boundaries
    albMin = ALBMINELSE
    albMax = ALBMAXELSE
    
    # Special conditions
    # For planets with M-type host star
    if teffStar >= 2300 and teffStar <=3800:
        albMin = ALBMINELSE
        albMax = ALBMAXM
    # For planets with G-type host star
    elif teffStar >= 5370 and teffStar <=5980:
        albmin = ALBMING     
        
    
    ###### Calculate F_OLR
    ### Albedo increments
    da = 0.01
    ### Eccentricity increments
    de = 0.01
    ### Sum of pofe (probability of eccentricity);
    ### is used to normalize the index value, later)
    pofeSum = 0
    ### How many instances of F_OLR meets the requirements for [...]
    ### the planet to have surface liquid water? Each instances would be [...]
    ### multiplied by the probability of that value of F_OLR from occuring
    habFact = 0
    ### Incoming stellar radiation (instellation)
    flux0 = luminosity/(16*math.pi*semiAxis*semiAxis)

    ### Calculate H
    a = albMin
    while a < albMax:
        e = eccLower
        while e < eccUpper:
            flux = flux0*(1-a)/math.sqrt(1-e*e)
            pofeSum = pofeSum + mpofe(e, ecc, eccUpRel)
            if flux < fMax and flux > MINFLUX:
                habFact = habFact + mpofe(e, ecc, eccUpRel)
            e = e + de
        a = a + da   
    
    if ecc > 0.8:
        H = 0.0
    elif pofeSum != 0:
        H = (habFact/pofeSum)*pRocky
    else: # for error case
        H = 0.0
    habIndexList.extend([exoName, ",", H])
        
    print(exoName, H)
    
    

mu1:  0.0
mu2:  0.0
sigma:  0.0
51 Eri b 0.0
mu1:  1.8792
mu2:  2.231
sigma:  0.11726666666666663
55 Cnc e 0.0
mu1:  2.2038
mu2:  2.6189
sigma:  0.1383666666666666
BD +20 594 b 0.0
mu1:  0.0
mu2:  0.0
sigma:  0.0
CoRoT-1 b 0.0
mu1:  0.0
mu2:  0.0
sigma:  0.0
CoRoT-10 b 0.0
mu1:  0.0
mu2:  0.0
sigma:  0.0
CoRoT-11 b 0.0
mu1:  0.0
mu2:  0.0
sigma:  0.0
CoRoT-12 b 0.0
mu1:  0.0
mu2:  0.0
sigma:  0.0
CoRoT-13 b 0.0
mu1:  0.0
mu2:  0.0
sigma:  0.0
CoRoT-14 b 0.0
mu1:  3.2236000000000002
mu2:  3.8786
sigma:  0.21833333333333327
CoRoT-16 b 0.0
mu1:  0.0
mu2:  0.0
sigma:  0.0
CoRoT-17 b 0.0
mu1:  0.0
mu2:  0.0
sigma:  0.0
CoRoT-18 b 0.0
mu1:  0.0
mu2:  0.0
sigma:  0.0
CoRoT-19 b 0.0
mu1:  0.0
mu2:  0.0
sigma:  0.0
CoRoT-2 b 0.0
mu1:  0.0
mu2:  0.0
sigma:  0.0
CoRoT-20 b 0.0
mu1:  2.0566
mu2:  2.442
sigma:  0.12846666666666673
CoRoT-22 b 0.0
mu1:  0.0
mu2:  0.0
sigma:  0.0
CoRoT-23 b 0.0
mu1:  0.02029
mu2:  0.02963
sigma:  0.003113333333333334
CoRoT-24 b 0.0
mu1:  2.4652000000000003
mu2:  2.922

C:\Users\Zulfa\AppData\Local\Temp\ipykernel_7380\3181247636.py:72: RuntimeWarning: divide by zero encountered in double_scalars
  fMax = A*SB*(LH2O/(RGAS*math.log(pStar*math.sqrt(K0/(2*PLINE*surfGrav)))))**4


 0.0
sigma:  0.0
CoRoT-3 b 0.0
mu1:  3.2782
mu2:  3.9614000000000003
sigma:  0.22773333333333343
CoRoT-4 b 0.0
mu1:  3.1836
mu2:  3.8211
sigma:  0.2124999999999999
CoRoT-5 b 0.0
mu1:  0.0
mu2:  0.0
sigma:  0.0
CoRoT-6 b 0.0
mu1:  1.5936000000000001
mu2:  1.8985
sigma:  0.10163333333333331
CoRoT-7 b 0.0
mu1:  2.9032
mu2:  3.4642999999999997
sigma:  0.18703333333333325
CoRoT-8 b 0.0
mu1:  0.0
mu2:  3.9924999999999997
sigma:  1.3308333333333333
CoRoT-9 b 0.0
mu1:  1.6722000000000001
mu2:  1.9914999999999998
sigma:  0.10643333333333323
EPIC 211822797 b 0.0
mu1:  2.4474
mu2:  2.9114
sigma:  0.15466666666666665
EPIC 211945201 b 0.0
mu1:  1.2212
mu2:  1.4594
sigma:  0.07939999999999998
GJ 1132 b 0.0
mu1:  1.7676
mu2:  2.096
sigma:  0.10946666666666667
GJ 1214 b 0.0
mu1:  2.1302
mu2:  2.531
sigma:  0.13360000000000008
GJ 3470 b 0.0
mu1:  2.376
mu2:  2.8142
sigma:  0.1460666666666667
GJ 436 b 0.0
mu1:  1.4868
mu2:  1.774
sigma:  0.09573333333333338
GJ 9827 b 0.0
mu1:  1.3559
mu2:  1.6232
sigma:

C:\Users\Zulfa\AppData\Local\Temp\ipykernel_7380\3181247636.py:109: RuntimeWarning: divide by zero encountered in double_scalars
  flux0 = luminosity/(16*math.pi*semiAxis*semiAxis)


mu2:  2.879
sigma:  0.14979999999999993
Kepler-1321 b 0.0
mu1:  1.7202000000000002
mu2:  2.0383
sigma:  0.10603333333333327
Kepler-1321 c 0.0
mu1:  1.5348
mu2:  1.8295000000000001
sigma:  0.0982333333333334
Kepler-1322 b 0.0
mu1:  1.5444
mu2:  1.841
sigma:  0.09886666666666666
Kepler-1323 b 0.0
mu1:  1.5348
mu2:  1.8295000000000001
sigma:  0.0982333333333334
Kepler-1324 b 0.0
mu1:  1.8513000000000002
mu2:  2.198
sigma:  0.1155666666666666
Kepler-1325 b 0.0
mu1:  2.2584999999999997
mu2:  2.6843
sigma:  0.14193333333333338
Kepler-1326 b 0.0
mu1:  1.5738
mu2:  1.8755000000000002
sigma:  0.1005666666666667
Kepler-1327 b 0.0
mu1:  0.99687
mu2:  1.2048999999999999
sigma:  0.06934333333333327
Kepler-1328 b 0.0
mu1:  1.6626
mu2:  1.9798
sigma:  0.1057333333333333
Kepler-1329 b 0.0
mu1:  1.5639
mu2:  1.864
sigma:  0.10003333333333335
Kepler-133 b 0.0
mu1:  1.8327
mu2:  2.187
sigma:  0.11809999999999994
Kepler-133 c 0.0
mu1:  1.5444
mu2:  1.841
sigma:  0.09886666666666666
Kepler-1330 b 0.0
mu1: 

In [7]:
# print(sum(habIndexList)/len(habIndexList)) #0.002815606263929704
print(habIndexList)

['51 Eri b', ',', 0.0, '55 Cnc e', ',', 0.0, 'BD +20 594 b', ',', 0.0, 'CoRoT-1 b', ',', 0.0, 'CoRoT-10 b', ',', 0.0, 'CoRoT-11 b', ',', 0.0, 'CoRoT-12 b', ',', 0.0, 'CoRoT-13 b', ',', 0.0, 'CoRoT-14 b', ',', 0.0, 'CoRoT-16 b', ',', 0.0, 'CoRoT-17 b', ',', 0.0, 'CoRoT-18 b', ',', 0.0, 'CoRoT-19 b', ',', 0.0, 'CoRoT-2 b', ',', 0.0, 'CoRoT-20 b', ',', 0.0, 'CoRoT-22 b', ',', 0.0, 'CoRoT-23 b', ',', 0.0, 'CoRoT-24 b', ',', 0.0, 'CoRoT-24 c', ',', 0.0, 'CoRoT-25 b', ',', 0.0, 'CoRoT-26 b', ',', 0.0, 'CoRoT-27 b', ',', 0.0, 'CoRoT-28 b', ',', 0.0, 'CoRoT-29 b', ',', 0.0, 'CoRoT-3 b', ',', 0.0, 'CoRoT-4 b', ',', 0.0, 'CoRoT-5 b', ',', 0.0, 'CoRoT-6 b', ',', 0.0, 'CoRoT-7 b', ',', 0.0, 'CoRoT-8 b', ',', 0.0, 'CoRoT-9 b', ',', 0.0, 'EPIC 211822797 b', ',', 0.0, 'EPIC 211945201 b', ',', 0.0, 'GJ 1132 b', ',', 0.0, 'GJ 1214 b', ',', 0.0, 'GJ 3470 b', ',', 0.0, 'GJ 436 b', ',', 0.0, 'GJ 9827 b', ',', 0.0, 'GJ 9827 c', ',', 0.0, 'GJ 9827 d', ',', 0.0, 'HAT-P-1 b', ',', 0.0, 'HAT-P-11 b', ',', 0.0,

In [11]:
with open('out.txt','w') as f:
    i = 1
    for a in habIndexList:
        if i == 3:
            print(a, file=f)
            i = 1
        else:
            print(a, file=f, end="")
            i += 1